In [1]:
import pandas as pd
import plotly.express as px
from datetime import datetime

from summer.utils import ref_times_to_dti

from autumn.models.sm_sir.stratifications.agegroup import convert_param_agegroups
from autumn.settings.constants import GOOGLE_MOBILITY_LOCATIONS
from autumn.core.inputs.mobility.queries import get_mobility_data
from autumn.models.sm_sir.mixing_matrix.macrodistancing import weight_mobility_data
from autumn.core import inputs
from autumn.settings.constants import COVID_BASE_DATETIME
from autumn.core.utils.display import pretty_print
from autumn.core.project import get_project
from autumn.projects.sm_sir.australia.northern_territory.project import get_ts_date_indexes
from autumn.core.inputs.social_mixing.build_synthetic_matrices import build_synthetic_matrices

In [2]:
pd.options.plotting.backend = "plotly"

In [3]:
region = "northern_territory"
p = get_project("sm_sir", region, reload=True)
baseline_params = p.param_set.baseline

Grab a few reused parameters

In [4]:
country = baseline_params["country"]["iso3"]
sub_region = baseline_params["population"]["region"]
data_year = baseline_params["population"]["year"]
ref_matrix_country = baseline_params["ref_mixing_iso3"]

### Population

In [6]:
modelled_age_groups = baseline_params["age_groups"]
age_pops = pd.Series(
    inputs.get_population_by_agegroup(
        modelled_age_groups,
        country,
        sub_region,
        data_year,
    ),
    index=modelled_age_groups
)
print(f"Total modelled population is {age_pops.sum() / 1e3} thousand")
print(f"Modelled age groups are {modelled_age_groups}")
age_pops.plot.bar()

Total modelled population is 246.143 thousand
Modelled age groups are [0, 15, 25, 50, 70]


### Mixing matrix

In [7]:
print(f"Target country: {country}")
print(f"Target sub-region: {sub_region}")
print(f"Proxy country: {ref_matrix_country}")
print("Always age-adjusted to target population under current SM-SIR code")

Target country: AUS
Target sub-region: Northern Territory
Proxy country: GBR
Always age-adjusted to target population under current SM-SIR code


Note that in the Victoria Nature Communications paper, we used the UK Polymod matrices in our base analysis as the reference matrices, and the Belgium Polymod matrices for our sensitivity analysis.

In [8]:
mixing_matrices = build_synthetic_matrices(
    country,
    ref_matrix_country,
    [int(age) for age in modelled_age_groups],
    True,
    sub_region,
)

In [9]:
print(f"Total daily contacts for each age group is {mixing_matrices['all_locations'].sum(axis=1)}")
px.imshow(mixing_matrices["all_locations"])

Total daily contacts for each age group is [14.33065545 14.17436073 12.54417699  9.46108668  6.28934597]


### Mobility

In [10]:
mob_df, google_mobility_days = get_mobility_data(
    country, 
    sub_region,
    COVID_BASE_DATETIME,
)
mob_dates_df = mob_df.copy()
mob_dates_df.index = mob_df["date"]

In [11]:
mob_dates_df[GOOGLE_MOBILITY_LOCATIONS].plot(title="Available raw mobility fields")

In [12]:
mobility_map_req = baseline_params["mobility"]["google_mobility_locations"]
model_loc_mobility_values = weight_mobility_data(
    mob_df, 
    mobility_map_req,
)
print("Requested mapping:")
pretty_print(mobility_map_req)
model_loc_mobility_values.index = mob_df["date"]
model_loc_mobility_values.plot(title="Mobility model implementation")

Requested mapping:
{ 'other_locations': { 'grocery_and_pharmacy': 0.333,
                       'parks': 0.0,
                       'retail_and_recreation': 0.333,
                       'transit_stations': 0.334},
  'work': {'workplaces': 1.0}}


### Age adjustments
#### Susceptibility

In [13]:
age_params = baseline_params["age_stratification"]

In [14]:
suscept_req = age_params["susceptibility"]
suscept_adjs = convert_param_agegroups(
    country,
    sub_region, 
    age_params["susceptibility"],
    [str(a) for a in modelled_age_groups],
)
suscept_req_df = pd.Series(suscept_req)

In [15]:
comparison_df = pd.DataFrame({
    "modelled": suscept_req_df,
    "data": suscept_adjs,
})
comparison_df.plot()

#### Symptomatic proportion

In [16]:
sympt_req = age_params["prop_symptomatic"]
sympt_adjs = convert_param_agegroups(
    country,
    sub_region, 
    age_params["prop_symptomatic"],
    [str(a) for a in modelled_age_groups],
)
sympt_req_df = pd.Series(sympt_req)

In [17]:
comparison_df = pd.DataFrame({
    "modelled": sympt_req_df,
    "data": sympt_adjs,
})
comparison_df.plot()

In [18]:
pd.Series(age_params["cfr"]["values"]).plot(title="Case fatality proportion")

In [19]:
pd.Series(age_params["prop_hospital"]["values"]).plot(title="Hospitalised proportion")

### Calibration targets

In [20]:
ts_set_dates = get_ts_date_indexes(p.ts_set, COVID_BASE_DATETIME)

In [21]:
ts_set_dates["notifications"].plot()